<center>
<img src='./img/nsidc_logo.png'/>

# **IceFlow**
### Point Cloud Data Access
</center>

---

# Accessing Data using the IceFlow API

This notebook shows how to access data programmatically using the *IceFlow* API and describes the parameters in more detail.<br>


## API Parameters:

### Selecting Data Sets

* The ***datasets*** parameter allows to list all data set of interest. See the [introduction notebook](./0_introduction.ipynb) section 4 for more information on the available data sets.</br>
Example: `'datasets': ['GLAH06', 'ATL06']`

### Choosing Optional Corrections

* The optional ***ITRF*** parameter allows you to choose an ITRF reference to which the data will be transformed via the published ITRF transformation parameters. This parameter is optional but must be used if you want to specify an epoch. Available values are: **ITRF2000, ITRF2008, ITRF2014**</br>
Example: `'ITRF': '2014'`
* The ***epoch*** parameter is optional and entered in decimal years to which the data will be transformed via the ITRF Plate Motion Model corresponding to ITRF. This parameter can only be used if the ***ITRF*** parameter is specified and set to either 2008 or 20014, as only ITRF2008 and ITRF2014 have a plate motion model. </br>
Example: `'epoch': '2014.1'` (This specifies January 2014.)

### Setting Temporal Extend

You can set the range of dates over which to return data using the ***start*** and ***end*** parameters:
* The ***start*** parameter accepts UTF datetime or simple YYYY-mm-dd formatted values</br>
Example: `'start': '2003-01-01'`
* The ***end*** parameter accepts UTF datetime or simple YYYY-mm-dd formatted values</br>
Example: `'end': '2019-01-01'`

### Setting Spatial Extend

You can use one of the two available options (***polygon***,***bbox***) to make your spatial selection:

* The ***Polygon*** parameter allows you to enter a counterclockwise closed array of latitude and longitude values ending with the same coordinate as the starting pair.</br>
Example: `'polygon'='-50.5811,69.4995,-49.3726,69.7632,-48.6035,69.8291,-47.8345,69.4995,-47.6587,68.8843,-48.6255,68.3130,-49.8077,69.0601,-50.4449,68.9941,-50.5811,69.4995'` 

* The ***bbox*** parameter allows you to choose a WGS84 bounding box with min_lon, min_lat, max_lon, max_lat values.</br>
Example: `'bbox': '-107.4515,-75.3695,-105.3794,-74.4563'` 

The following code is an example on how to order data with the *IceFlow* API, but first you have to log into EarthData:
### NASA's Earthdata Credentials

To access data using the *IceFlow* library it is necessary to log into [Earthdata Login](https://urs.earthdata.nasa.gov/). We showed you how to do this with a user interface in the [introduction](0_introduction.ipynb) and the [widget](1_widget.ipynb) notebooks. Here we show you how to do that programmatically. For that you have to change the values for `'username'`,`'password'`, and `'email'` to your personal Earthdata credentials before you run the next cell. 

In [ ]:
# We import our API client and authenticate with our NASA Earthdata credentials
from iceflow.client import IceflowClient

ifc = IceflowClient()

if ifc.authenticate('username', 'password', 'email') is not None:
    print('Authenticated with NASA Earthdata')
else:
    print('Login failed')

In [ ]:
# We use IceFlow to query CMR and get an idea of how many granules we'll be sub-setting.

# Thwaites glacier ICESat/GLAS and ICESat-2
my_params ={
    'datasets': ['GLAH06', 'ATL06'],
    'ITRF': '2014',
    'epoch': '2014.1',
    'start': '2003-01-01',
    'end': '2019-01-01',
    'bbox': '-107.4515,-75.3695,-105.3794,-74.4563'
}
granules = ifc.query_cmr(my_params)

In [ ]:
# Now we order the data, this is an asynchronous process. 
# Asynchronous downloads allow concurrent requests to be queued and processed without the need for a continuous connection. 
# The maximum granule count per asynchronous request is 2000. When the order is complete
# Now you can proceed to download the IceFlow data granule.
orders = ifc.place_data_orders(my_params)
orders

In [ ]:
# To check the order, you can run this cell:
for order in orders:
    status = ifc.check_order_status(order)
    print(status)

In [ ]:
#Once all orders are complete you can download the data:
for order in orders:
    status = ifc.check_order_status(order)
    if status['status'] == 'COMPLETE':
        ifc.download_order(order)        